In [1]:
import os
import sys
import time
import json
from os.path import abspath, dirname
import pandas as pd
import numpy as np
import datetime
from IPython.display import display, clear_output

# current_dir = abspath(dirname(__file__))
current_dir = abspath(dirname(""))
project_path = abspath(dirname(dirname(current_dir)))
sys.path.append(project_path + "/TradeStreamEngine/StreamEngine")


import lookups
from synthesis import *
from flow import * 

In [2]:
class binance_flow():
    
    def __init__ (self, level_size):
        self.level_size = level_size
        self.spot_axis = {
            "books" : {
                "btcusdt" : booksflow('binance', 'btc_usdt', 'perpetual', level_size, lookups.binance_depth_lookup),
                "btcfdusd" : booksflow('binance', 'btc_fdusd', 'perpetual', level_size, lookups.binance_depth_lookup),
            },
            "trades": {
                "btcusdt" : tradesflow('binance', 'btc_usdt', 'perpetual', level_size, lookups.binance_trades_lookup),
                "btcfdusd" : tradesflow('binance', 'btc_fdusd', 'perpetual', level_size, lookups.binance_trades_lookup),
            },
        }
        self.perpetual_axis= {
            "books" : {
                "btcusdt" : booksflow('binance', 'btc_usdt', 'perpetual', level_size, lookups.binance_depth_lookup),
                "btcusd" : booksflow('binance', 'btc_usd', 'perpetual', level_size, lookups.binance_depth_lookup),
            },
            "trades": {
                "btcusdt" : tradesflow('binance', 'btc_usdt', 'perpetual', level_size, lookups.binance_trades_lookup),
                "btcusd" : tradesflow('binance', 'btc_usd', 'perpetual', level_size, lookups.binance_trades_lookup),
            },
            "oifunding" : {
                "btcusdt" : oiFundingflow('binance', 'btc_usdt', 'perpetual', level_size, lookups.binance_OI_lookup, lookups.binance_funding_lookup),
                "btcusd" : oiFundingflow('binance', 'btc_usd', 'perpetual', level_size, lookups.binance_OI_lookup, lookups.binance_funding_lookup),
            },
            "liquidations" : {
                "btcusdt" : liquidationsflow('binance', 'btc_usdt', 'perpetual', level_size, lookups.binance_liquidations_lookup),
                "btcusd" : liquidationsflow('binance', 'btc_usd', 'perpetual', level_size, lookups.binance_liquidations_lookup),
            },
            "indicators" : {
                "btcusdt_TTA" : indicatorflow('binance', 'btc_usdt', 'perpetual', "TTA", lookups.binance_GTA_TTA_TTP_lookup),
                "btcusdt_TTP" : indicatorflow('binance', 'btc_usdt', 'perpetual', "TTP", lookups.binance_GTA_TTA_TTP_lookup),
                "btcusdt_GTA" : indicatorflow('binance', 'btc_usdt', 'perpetual', "GTA", lookups.binance_GTA_TTA_TTP_lookup),
                "btcusd_TTA" : indicatorflow('binance', 'btc_usd', 'perpetual', "TTA", lookups.binance_GTA_TTA_TTP_lookup),
                "btcusd_TTP" : indicatorflow('binance', 'btc_usd', 'perpetual', "TTP", lookups.binance_GTA_TTA_TTP_lookup),
                "btcusd_GTA" : indicatorflow('binance', 'btc_usd', 'perpetual', "GTA", lookups.binance_GTA_TTA_TTP_lookup),
            }
        }

    def add_d_spot_usdt(self, data):
        self.spot_axis["books"]["btcusdt"].update_books(data)

    def add_d_spot_fdusd(self, data):
        self.spot_axis["books"]["btcfdusd"].update_books(data)

    def add_t_spot_usdt(self, data):
        self.spot_axis["trades"]["btcusdt"].input_trades(data)

    def add_t_spot_fdusd(self, data):
        self.spot_axis["trades"]["btcfdusd"].input_trades(data)

    def add_d_perp_usdt(self, data):
        self.perpetual_axis["books"]["btcusdt"].update_books(data)

    def add_d_perp_usd(self, data):
        self.perpetual_axis["books"]["btcusd"].update_books(data)

    def add_t_perp_usdt(self, data):
        self.perpetual_axis["trades"]["btcusdt"].input_trades(data)

    def add_t_perp_usd(self, data):
        self.perpetual_axis["trades"]["btcusd"].input_trades(data)

    def add_oi_usdt(self, data):
        self.perpetual_axis["oifunding"]["btcusdt"].input_oi(data)

    def add_f_usdt(self, data):
        self.perpetual_axis["oifunding"]["btcusdt"].input_funding(data)

    def add_oi_usd(self, data):
        self.perpetual_axis["oifunding"]["btcusd"].input_oi(data)

    def add_f_usd(self, data):
        self.perpetual_axis["oifunding"]["btcusd"].input_funding(data)

    def add_l_usdt(self, data):
        self.perpetual_axis["liquidations"]["btcusdt"].input_liquidations(data)

    def add_l_usd(self, data):
        self.perpetual_axis["liquidations"]["btcusd"].input_liquidations(data)

    def add_tta_usdt(self, data):
        self.perpetual_axis["indicators"]["btcusdt_TTA"].input_binance_gta_tta_ttp(data)

    def add_tta_usd(self, data):
        self.perpetual_axis["indicators"]["btcusd_TTA"].input_binance_gta_tta_ttp(data)

    def add_ttp_usdt(self, data):
        self.perpetual_axis["indicators"]["btcusdt_TTP"].input_binance_gta_tta_ttp(data)

    def add_ttp_usd(self, data):
        self.perpetual_axis["indicators"]["btcusd_TTP"].input_binance_gta_tta_ttp(data)

    def add_gta_usdt(self, data):
        self.perpetual_axis["indicators"]["btcusdt_GTA"].input_binance_gta_tta_ttp(data)

    def add_gta_usd(self, data):
        self.perpetual_axis["indicators"]["btcusd_GTA"].input_binance_gta_tta_ttp(data)


    # Testing
    
    @classmethod
    def create_class(cls, level_size=20):
        return cls(level_size)

    def input_from_json(self):
        """
            json files should be in the directory data/
            Add your own logic
        """
        files = [
            "binance_btcusdt_spot_depth", "binance_btcfdusd_spot_depth", "binance_btcusdt_spot_trades", "binance_btcfdusd_spot_trades",
            "binance_btcusdt_perpetual_depth", "binance_btcusd_perpetual_depth", "binance_btcusdt_perpetual_trades", "binance_btcusd_perpetual_trades",
            "binance_btcusdt_perpetual_fundingRate", "binance_btcusd_perpetual_fundingRate", "binance_btcusdt_perpetual_OI", "binance_btcusd_perpetual_OI",
            "binance_btcusdt_perpetual_liquidations", "binance_btcusd_perpetual_liquidations", "binance_btcusdt_perpetual_TTA", "binance_btcusd_perpetual_TTA",
            "binance_btcusdt_perpetual_TTP", "binance_btcusd_perpetual_TTP", "binance_btcusdt_perpetual_GTA", "binance_btcusd_perpetual_GTA",
            ]

        functions_list = [
            self.add_d_spot_usdt, self.add_d_spot_fdusd, self.add_t_spot_usdt, self.add_t_spot_fdusd,
            self.add_d_perp_usdt, self.add_d_perp_usd, self.add_t_perp_usdt, self.add_t_perp_usd,
            self.add_f_usdt, self.add_f_usd, self.add_oi_usdt, self.add_oi_usd,
            self.add_l_usdt, self.add_l_usd, self.add_tta_usdt, self.add_tta_usd,
            self.add_ttp_usdt, self.add_ttp_usd, self.add_gta_usdt, self.add_gta_usd
        ]
        v = {
            x : { 
            "data" : json.load(open("".join(["data/", x, ".json"]))),
            "lookup" : f,
            } 
            for x, f in zip(files, functions_list)
            }

        for key in v.keys():
            for d in v[key]["data"]:
                d = json.dumps(d)
                v[key]["lookup"](d)


    def test_empty_dataframes(self):
        """
            My json files liquidaitons is empty because there is no data, better try with trades
        """

        for flow in self.spot_axis.keys():
            for instrument in self.spot_axis[flow].keys():
                if flow != "indicators":
                    if flow in ["books", "oifunding"]:
                        s = self.spot_axis[flow][instrument].snapshot
                    if flow in ["trades", "liquidations"]:
                        s = self.spot_axis[flow][instrument].snapshot_total
                    try:
                        if s.empty:
                            print(f"perpetual_axis, {flow}, {instrument}, is empty")
                    except:
                        print(f"perpetual_axis, {flow}, {instrument}, is NoneType")
                if flow == "indicators":
                    if not bool( self.spot_axis[flow][instrument].data):
                        print(f"spot_axis, {flow}, {instrument}, is empty")


        for flow in self.perpetual_axis.keys():
            for instrument in self.perpetual_axis[flow].keys():
                if flow != "indicators":
                    if flow in ["books", "oifunding"]:
                        s = self.perpetual_axis[flow][instrument].snapshot
                    if flow in ["trades", "liquidations"]:
                        s = self.perpetual_axis[flow][instrument].snapshot_total
                    try:
                        if s.empty:
                            print(f"perpetual_axis, {flow}, {instrument}, is empty")
                    except:
                        print(f"perpetual_axis, {flow}, {instrument}, is NoneType")
                    
                if flow == "indicators":
                    if not bool( self.perpetual_axis[flow][instrument].data):
                        print(f"perpetual_axis, {flow}, {instrument}, is empty")

        time.sleep(5)

        # # Observe snapshots for any discrepancy
        # for flow in self.spot_axis.keys():
        #     for instrument in self.spot_axis[flow].keys():
        #         if flow != "indicators":
        #             if flow in ["books", "oifunding"]:
        #                 s = self.spot_axis[flow][instrument].snapshot
        #             if flow in ["trades", "liquidations"]:
        #                 s = self.spot_axis[flow][instrument].snapshot_total
        #             try:
        #                 display(s)
        #             except:
        #                 pass
        #         if flow == "indicators":
        #             s = self.spot_axis[flow][instrument].data
        #             if len(s) != 0:
        #                 print(s)
        #         time.sleep(5)
        #         clear_output(wait=True)

        # for flow in self.perpetual_axis.keys():
        #     for instrument in self.perpetual_axis[flow].keys():
        #         if flow != "indicators":
        #             if flow in ["books", "oifunding"]:
        #                 s = self.perpetual_axis[flow][instrument].snapshot
        #             if flow in ["trades", "liquidations"]:
        #                 s = self.perpetual_axis[flow][instrument].snapshot_total
        #             try:
        #                 display(s)
        #             except:
        #                 pass
        #         if flow == "indicators":
        #             s = self.perpetual_axis[flow][instrument].data
        #             if not bool(s):
        #                 print(s)
        #         time.sleep(5)
        #         clear_output(wait=True)


In [9]:
class okx_flow():
    
    def __init__ (self, level_size):
        self.level_size = level_size
        self.spot_axis = {
            "books" : {
                "btcusdt" : booksflow('okx', 'btc_usdt', 'perpetual', level_size, lookups.okx_depth_lookup),
            },
            "trades": {
                "btcusdt" : tradesflow('okx', 'btc_usdt', 'perpetual', level_size, lookups.okx_trades_lookup),
            },
        }
        self.perpetual_axis= {
            "books" : {
                "btcusdt" : booksflow('okx', 'btc_usdt', 'perpetual', level_size, lookups.okx_depth_lookup),
                "btcusd" : booksflow('okx', 'btc_usd', 'perpetual', level_size, lookups.okx_depth_lookup),
            },
            "trades": {
                "btcusdt" : tradesflow('okx', 'btc_usdt', 'perpetual', level_size, lookups.okx_trades_lookup),
                "btcusd" : tradesflow('okx', 'btc_usd', 'perpetual', level_size, lookups.okx_trades_lookup),
            },
            "oifunding" : {
                "btcusdt" : oiFundingflow('okx', 'btc_usdt', 'perpetual', level_size, lookups.okx_OI_lookup, lookups.okx_funding_lookup),
                "btcusd" : oiFundingflow('okx', 'btc_usd', 'perpetual', level_size, lookups.okx_OI_lookup, lookups.okx_funding_lookup),
            },
            "liquidations" : {
                "btc" : liquidationsflow('okx', 'btc_usdt', 'perpetual', level_size, lookups.okx_liquidations_lookup),
            },
            "indicators" : {
                "GTA" : indicatorflow('okx', 'btc_usdt', 'perpetual', "TTA", lookups.okx_GTA_lookup),
            }
        }

    def add_d_spot_usdt(self, data):
        self.spot_axis["books"]["btcusdt"].update_books(data)

    def add_t_spot_usdt(self, data):
        self.spot_axis["trades"]["btcusdt"].input_trades(data)

    def add_d_perp_usdt(self, data):
        self.perpetual_axis["books"]["btcusdt"].update_books(data)

    def add_d_perp_usd(self, data):
        self.perpetual_axis["books"]["btcusd"].update_books(data)

    def add_t_perp_usdt(self, data):
        self.perpetual_axis["trades"]["btcusdt"].input_trades(data)

    def add_t_perp_usd(self, data):
        self.perpetual_axis["trades"]["btcusd"].input_trades(data)

    def add_oi_usdt(self, data):
        self.perpetual_axis["oifunding"]["btcusdt"].input_oi(data)

    def add_f_usdt(self, data):
        self.perpetual_axis["oifunding"]["btcusdt"].input_funding(data)

    def add_oi_usd(self, data):
        self.perpetual_axis["oifunding"]["btcusd"].input_oi(data)

    def add_f_usd(self, data):
        self.perpetual_axis["oifunding"]["btcusd"].input_funding(data)

    def add_l(self, data):
        self.perpetual_axis["liquidations"]["btc"].input_liquidations(data)

    def add_gta(self, data):
        self.perpetual_axis["indicators"]["GTA"].input_okx_gta(data)


    # Testing
    
    @classmethod
    def create_class(cls, level_size=20):
        return cls(level_size)

    def input_from_json(self):
        """
            json files should be in the directory data/
            Add your own logic
        """
        files = [
            "okx_btcusdt_spot_depth", "okx_btcusdt_spot_trades", 
            "okx_btcusdt_perpetual_depth", "okx_btcusd_perpetual_depth", "okx_btcusdt_perpetual_trades", "okx_btcusd_perpetual_trades",
            "okx_btcusdt_perpetual_fundingRate", "okx_btcusd_perpetual_fundingRate", "okx_btcusdt_perpetual_OI", "okx_btcusd_perpetual_OI",
            "okx_btcusdt_perpetual_liquidations", "okx_btcusd_perpetual_GTA",
            ]

        functions_list = [
            self.add_d_spot_usdt,  self.add_t_spot_usdt, 
            self.add_d_perp_usdt, self.add_d_perp_usd, self.add_t_perp_usdt, self.add_t_perp_usd,
            self.add_f_usdt, self.add_f_usd, self.add_oi_usdt, self.add_oi_usd,
            self.add_l, self.add_gta
        ]
        v = {
            x : { 
            "data" : json.load(open("".join(["data/", x, ".json"]))),
            "lookup" : f,
            } 
            for x, f in zip(files, functions_list)
            }

        for key in v.keys():
            for d in v[key]["data"]:
                d = json.dumps(d)
                v[key]["lookup"](d)


    def test_empty_dataframes(self):
        """
            My json files liquidaitons is empty because there is no data, better try with trades
        """

        for flow in self.spot_axis.keys():
            for instrument in self.spot_axis[flow].keys():
                if flow != "indicators":
                    if flow in ["books", "oifunding"]:
                        s = self.spot_axis[flow][instrument].snapshot
                    if flow in ["trades", "liquidations"]:
                        s = self.spot_axis[flow][instrument].snapshot_total
                    try:
                        if s.empty:
                            print(f"perpetual_axis, {flow}, {instrument}, is empty")
                    except:
                        print(f"perpetual_axis, {flow}, {instrument}, is NoneType")
                if flow == "indicators":
                    if not bool( self.spot_axis[flow][instrument].data):
                        print(f"spot_axis, {flow}, {instrument}, is empty")


        for flow in self.perpetual_axis.keys():
            for instrument in self.perpetual_axis[flow].keys():
                if flow != "indicators":
                    if flow in ["books", "oifunding"]:
                        s = self.perpetual_axis[flow][instrument].snapshot
                    if flow in ["trades", "liquidations"]:
                        s = self.perpetual_axis[flow][instrument].snapshot_total
                    try:
                        if s.empty:
                            print(f"perpetual_axis, {flow}, {instrument}, is empty")
                    except:
                        print(f"perpetual_axis, {flow}, {instrument}, is NoneType")
                    
                if flow == "indicators":
                    if not bool( self.perpetual_axis[flow][instrument].data):
                        print(f"perpetual_axis, {flow}, {instrument}, is empty")


    def test_dataframes(self):
        # Observe snapshots for any discrepancy
        for flow in self.spot_axis.keys():
            for instrument in self.spot_axis[flow].keys():
                if flow != "indicators":
                    if flow in ["books", "oifunding"]:
                        s = self.spot_axis[flow][instrument].snapshot
                    if flow in ["trades", "liquidations"]:
                        s = self.spot_axis[flow][instrument].snapshot_total
                    try:
                        display(s)
                    except:
                        pass
                if flow == "indicators":
                    s = self.spot_axis[flow][instrument].data
                    if len(s) != 0:
                        print(s)
                time.sleep(5)
                clear_output(wait=True)

        for flow in self.perpetual_axis.keys():
            for instrument in self.perpetual_axis[flow].keys():
                if flow != "indicators":
                    if flow in ["books", "oifunding"]:
                        s = self.perpetual_axis[flow][instrument].snapshot
                    if flow in ["trades", "liquidations"]:
                        s = self.perpetual_axis[flow][instrument].snapshot_total
                    try:
                        display(s)
                    except:
                        pass
                if flow == "indicators":
                    s = self.perpetual_axis[flow][instrument].data
                    if not bool(s):
                        print(s)
                time.sleep(5)
                clear_output(wait=True)


In [ ]:
import sys
import time
import json
from os.path import abspath, dirname
from IPython.display import display, clear_output

current_dir = abspath(dirname(__file__))
project_path = abspath(dirname(dirname(current_dir)))
sys.path.append(project_path + "/TradeStreamEngine/StreamEngineBase")

import lookups
import flow



class bybit_flow():
    
    def __init__ (self, level_size):
        self.level_size = level_size
        self.spot_axis = {
            "books" : {
                "btcusdt" : flow.booksflow('bybit', 'btc_usdt', 'perpetual', level_size, lookups.bybit_depth_lookup),
                "btcusdc" : flow.booksflow('bybit', 'btc_usdc', 'perpetual', level_size, lookups.binance_depth_lookup),
            },
            "trades": {
                "btcusdt" : flow.tradesflow('bybit', 'btc_usdt', 'perpetual', level_size, lookups.bybit_trades_lookup),
                "btcusdc" : flow.tradesflow('bybit', 'btc_usdc', 'perpetual', level_size, lookups.bybit_trades_lookup),
            },
        }
        self.perpetual_axis= {
            "books" : {
                "btcusdt" : flow.booksflow('bybit', 'btc_usdt', 'perpetual', level_size, lookups.bybit_depth_lookup),
                "btcusd" : flow.booksflow('bybit', 'btc_usd', 'perpetual', level_size, lookups.bybit_depth_lookup),
            },
            "trades": {
                "btcusdt" : flow.tradesflow('bybit', 'btc_usdt', 'perpetual', level_size, lookups.bybit_trades_lookup),
                "btcusd" : flow.tradesflow('bybit', 'btc_usd', 'perpetual', level_size, lookups.bybit_trades_lookup),
            },
            "oifunding" : {
                "btcusdt" : flow.oiFundingflow('bybit', 'btc_usdt', 'perpetual', level_size, lookups.bybit_OI_funding_lookup),
                "btcusd" : flow.oiFundingflow('bybit', 'btc_usd', 'perpetual', level_size, lookups.bybit_OI_funding_lookup),
            },
            "liquidations" : {
                "btcusdt" : flow.liquidationsflow('bybit', 'btc_usdt', 'perpetual', level_size, lookups.bybit_liquidations_lookup),
                "btcusd" : flow.liquidationsflow('bybit', 'btc_usd', 'perpetual', level_size, lookups.bybit_liquidations_lookup),
            },
            "indicators" : {
                "btcusdt_GTA" : flow.indicatorflow('bybit', 'btc_usdt', 'perpetual', "TTA", lookups.bybit_GTA_lookup),
                "btcusd_GTA" : flow.indicatorflow('bybit', 'btc_usd', 'perpetual', "TTA", lookups.bybit_GTA_lookup)
            }
        }

    def add_d_spot_usdt(self, data):
        self.spot_axis["books"]["btcusdt"].update_books(data)

    def add_d_spot_usdc(self, data):
        self.spot_axis["books"]["btcusdc"].update_books(data)

    def add_t_spot_usdt(self, data):
        self.spot_axis["trades"]["btcusdt"].input_trades(data)

    def add_t_spot_usdc(self, data):
        self.spot_axis["trades"]["btcusdc"].input_trades(data)

    def add_d_perp_usdt(self, data):
        self.perpetual_axis["books"]["btcusdt"].update_books(data)

    def add_d_perp_usd(self, data):
        self.perpetual_axis["books"]["btcusd"].update_books(data)

    def add_t_perp_usdt(self, data):
        self.perpetual_axis["trades"]["btcusdt"].input_trades(data)

    def add_t_perp_usd(self, data):
        self.perpetual_axis["trades"]["btcusd"].input_trades(data)

    def add_oif_usdt(self, data):
        self.perpetual_axis["oifunding"]["btcusdt"].input_oi_funding(data)

    def add_oif_usd(self, data):
        self.perpetual_axis["oifunding"]["btcusd"].input_oi_funding(data)

    def add_l_usdt(self, data):
        self.perpetual_axis["liquidations"]["btcusdt"].input_liquidations(data)

    def add_l_usd(self, data):
        self.perpetual_axis["liquidations"]["btcusd"].input_liquidations(data)

    def add_gta_usdt(self, data):
        self.perpetual_axis["indicators"]["btcusdt_TTA"].input_bybit_gta(data)

    def add_gta_usd(self, data):
        self.perpetual_axis["indicators"]["btcusd_TTA"].input_bybit_gta(data)

    # Testing
    
    @classmethod
    def create_class(cls, level_size=20):
        return cls(level_size)

    def input_from_json(self):
        """
            json files should be in the directory data/
            Add your own logic
        """
        files = [
            "binance_btcusdt_spot_depth", "binance_btcfdusd_spot_depth", "binance_btcusdt_spot_trades", "binance_btcfdusd_spot_trades",
            "binance_btcusdt_perpetual_depth", "binance_btcusd_perpetual_depth", "binance_btcusdt_perpetual_trades", "binance_btcusd_perpetual_trades",
            "binance_btcusdt_perpetual_fundingRate", "binance_btcusd_perpetual_fundingRate", "binance_btcusdt_perpetual_OI", "binance_btcusd_perpetual_OI",
            "binance_btcusdt_perpetual_liquidations", "binance_btcusd_perpetual_liquidations", "binance_btcusdt_perpetual_TTA", "binance_btcusd_perpetual_TTA",
            "binance_btcusdt_perpetual_TTP", "binance_btcusd_perpetual_TTP", "binance_btcusdt_perpetual_GTA", "binance_btcusd_perpetual_GTA",
            ]

        functions_list = [
            self.add_d_spot_usdt, self.add_d_spot_fdusd, self.add_t_spot_usdt, self.add_t_spot_fdusd,
            self.add_d_perp_usdt, self.add_d_perp_usd, self.add_t_perp_usdt, self.add_t_perp_usd,
            self.add_f_usdt, self.add_f_usd, self.add_oi_usdt, self.add_oi_usd,
            self.add_l_usdt, self.add_l_usd, self.add_tta_usdt, self.add_tta_usd,
            self.add_ttp_usdt, self.add_ttp_usd, self.add_gta_usdt, self.add_gta_usd
        ]
        v = {
            x : { 
            "data" : json.load(open("".join(["data/", x, ".json"]))),
            "lookup" : f,
            } 
            for x, f in zip(files, functions_list)
            }

        for key in v.keys():
            for d in v[key]["data"]:
                d = json.dumps(d)
                v[key]["lookup"](d)


    def test_empty_dataframes(self):
        """
            My json files liquidaitons is empty because there is no data, better try with trades
        """

        for flow in self.spot_axis.keys():
            for instrument in self.spot_axis[flow].keys():
                if flow != "indicators":
                    if flow in ["books", "oifunding"]:
                        s = self.spot_axis[flow][instrument].snapshot
                    if flow in ["trades", "liquidations"]:
                        s = self.spot_axis[flow][instrument].snapshot_total
                    try:
                        if s.empty:
                            print(f"perpetual_axis, {flow}, {instrument}, is empty")
                    except:
                        print(f"perpetual_axis, {flow}, {instrument}, is NoneType")
                if flow == "indicators":
                    if not bool( self.spot_axis[flow][instrument].data):
                        print(f"spot_axis, {flow}, {instrument}, is empty")


        for flow in self.perpetual_axis.keys():
            for instrument in self.perpetual_axis[flow].keys():
                if flow != "indicators":
                    if flow in ["books", "oifunding"]:
                        s = self.perpetual_axis[flow][instrument].snapshot
                    if flow in ["trades", "liquidations"]:
                        s = self.perpetual_axis[flow][instrument].snapshot_total
                    try:
                        if s.empty:
                            print(f"perpetual_axis, {flow}, {instrument}, is empty")
                    except:
                        print(f"perpetual_axis, {flow}, {instrument}, is NoneType")
                    
                if flow == "indicators":
                    if not bool( self.perpetual_axis[flow][instrument].data):
                        print(f"perpetual_axis, {flow}, {instrument}, is empty")


    def test_dataframes(self):
        # Observe snapshots for any discrepancy
        for flow in self.spot_axis.keys():
            for instrument in self.spot_axis[flow].keys():
                if flow != "indicators":
                    if flow in ["books", "oifunding"]:
                        s = self.spot_axis[flow][instrument].snapshot
                    if flow in ["trades", "liquidations"]:
                        s = self.spot_axis[flow][instrument].snapshot_total
                    try:
                        display(s)
                    except:
                        pass
                if flow == "indicators":
                    s = self.spot_axis[flow][instrument].data
                    if len(s) != 0:
                        print(s)
                time.sleep(5)
                clear_output(wait=True)

        for flow in self.perpetual_axis.keys():
            for instrument in self.perpetual_axis[flow].keys():
                if flow != "indicators":
                    if flow in ["books", "oifunding"]:
                        s = self.perpetual_axis[flow][instrument].snapshot
                    if flow in ["trades", "liquidations"]:
                        s = self.perpetual_axis[flow][instrument].snapshot_total
                    try:
                        display(s)
                    except:
                        pass
                if flow == "indicators":
                    s = self.perpetual_axis[flow][instrument].data
                    if not bool(s):
                        print(s)
                time.sleep(5)
                clear_output(wait=True)

In [10]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

binance = okx_flow.create_class()
binance.input_from_json()
binance.test_empty_dataframes()

perpetual_axis, liquidations, btc, is NoneType


In [ ]:
perpetual_axis, liquidations, btcusdt, is NoneType
perpetual_axis, liquidations, btcusd, is NoneType

In [10]:
binance.perpetual_axis["indicators"]["btcusdt_GTA"].data

{'timestamp': '2024-01-30 09:50:00',
 'longAccount': 0.5549,
 'shortAccount': 0.4451,
 'ratio': 1.2467,
 'price': 43411.6}

In [7]:
class btcflow():

    def __init__ (self, level_size, pranges, expiry_windows):
        
        self.level_size = level_size
        self.pranges = pranges
        self.expiry_windows = expiry_windows

        self.spot_axis = {
            "books" : {
                "binance_btcusdt" : booksflow('binance', 'btc_usdt', 'perpetual', level_size, lookups.binance_depth_lookup),
                "binance_btcfdusd" : booksflow('binance', 'btc_fdusd', 'perpetual', level_size, lookups.binance_depth_lookup),
                "okx_btcusdt" : booksflow('okx', 'btc_usdt', 'perpetual', level_size, lookups.okx_depth_lookup),
                "bybit_btcusdt" : booksflow('bybit', 'btc_usdt', 'perpetual', level_size, lookups.bybit_depth_lookup),
                "bybit_btcusdc" : booksflow('bybit', 'btc_usdc', 'perpetual', level_size, lookups.bybit_depth_lookup),
                "coinbase_btcusd" : booksflow('bingx', 'btc_usd', 'perpetual', level_size, lookups.coinbase_depth_lookup),
                "bingx_btcusdt" : booksflow('bingx', 'btc_usdt', 'perpetual', level_size, lookups.bingx_depth_lookup),
                "bitget_btcusdt" : booksflow('bitget', 'btc_usdt', 'perpetual', level_size, lookups.bitget_depth_lookup),
                "gateio_btcusdt" : booksflow('gateio', 'btc_usdt', 'perpetual', level_size, lookups.gateio_depth_lookup),
                "htx_btcusdt" : booksflow('htx', 'btc_usdt', 'perpetual', level_size, lookups.htx_depth_lookup),
                "mexc_btcusdt" : booksflow('mexc', 'btc_usdt', 'perpetual', level_size, lookups.mexc_depth_lookup),
                "kucoin_btcusdt" : booksflow('kucoin', 'btc_usdt', 'perpetual', level_size, lookups.kucoin_depth_lookup),
            },
            "trades": {
                "binance_btcusdt" : tradesflow('binance', 'btc_usdt', 'perpetual', level_size, lookups.binance_trades_lookup),
                "binance_btcfdusd" : tradesflow('binance', 'btc_fdusd', 'perpetual', level_size, lookups.binance_trades_lookup),
                "okx_btcusdt" : tradesflow('okx', 'btc_usdt', 'perpetual', level_size, lookups.okx_trades_lookup),
                "bybit_btcusdt" : tradesflow('bybit', 'btc_usdt', 'perpetual', level_size, lookups.bybit_trades_lookup),
                "bybit_btcusdc" : tradesflow('bybit', 'btc_usdc', 'perpetual', level_size, lookups.bybit_trades_lookup),
                "coinbase_btcusd" : tradesflow('bingx', 'btc_usd', 'perpetual', level_size, lookups.coinbase_trades_lookup),
                "bingx_btcusdt" : tradesflow('bingx', 'btc_usdt', 'perpetual', level_size, lookups.bingx_trades_lookup),
                "bitget_btcusdt" : tradesflow('bitget', 'btc_usdt', 'perpetual', level_size, lookups.bitget_trades_lookup),
                "gateio_btcusdt" : tradesflow('gateio', 'btc_usdt', 'perpetual', level_size, lookups.gateio_trades_lookup),
                "htx_btcusdt" : tradesflow('htx', 'btc_usdt', 'perpetual', level_size, lookups.htx_trades_lookup),
                "mexc_btcusdt" : tradesflow('mexc', 'btc_usdt', 'perpetual', level_size, lookups.mexc_trades_lookup),
                "kucoin_btcusdt" : tradesflow('kucoin', 'btc_usdt', 'perpetual', level_size, lookups.kucoin_trades_lookup),
            },
        }
        self.perpetual_axis= {
            "books" : {
                "binance_btcusdt" : booksflow('binance', 'btc_usdt', 'perpetual', level_size, lookups.binance_depth_lookup),
                "binance_btcusd" : booksflow('binance', 'btc_usd', 'perpetual', level_size, lookups.binance_depth_lookup),
                "okx_btcusdt" : booksflow('okx', 'btc_usdt', 'perpetual', level_size, lookups.okx_depth_lookup),
                "okx_btcusd" : booksflow('okx', 'btc_usd', 'perpetual', level_size, lookups.okx_depth_lookup),
                "bybit_btcusdt" : booksflow('bybit', 'btc_usdt', 'perpetual', level_size, lookups.bybit_depth_lookup),
                "bybit_btcusd" : booksflow('bybit', 'btc_usd', 'perpetual', level_size, lookups.bybit_depth_lookup),
                "bingx_btcusdt" : booksflow('bingx', 'btc_usdt', 'perpetual', level_size, lookups.bingx_depth_lookup),
                "bitget_btcusdt" : booksflow('bitget', 'btc_usdt', 'perpetual', level_size, lookups.bitget_depth_lookup),
                "gateio_btcusdt" : booksflow('gateio', 'btc_usdt', 'perpetual', level_size, lookups.gateio_depth_lookup),
                "htx_btcusdt" : booksflow('htx', 'btc_usdt', 'perpetual', level_size, lookups.htx_depth_lookup),
                "mexc_btcusdt" : booksflow('mexc', 'btc_usdt', 'perpetual', level_size, lookups.mexc_depth_lookup),
                "kucoin_btcusdt" : booksflow('kucoin', 'btc_usdt', 'perpetual', level_size, lookups.kucoin_depth_lookup),
                "deribit_btcusd" : booksflow('deribit', 'btc_usd', 'perpetual', level_size, lookups.deribit_depth_lookup),
            },
            "trades": {
                "binance_btcusdt" : tradesflow('binance', 'btc_usdt', 'perpetual', level_size, lookups.binance_trades_lookup),
                "binance_btcusd" : tradesflow('binance', 'btc_usd', 'perpetual', level_size, lookups.binance_trades_lookup),
                "okx_btcusdt" : tradesflow('okx', 'btc_usdt', 'perpetual', level_size, lookups.okx_trades_lookup),
                "okx_btcusd" : tradesflow('okx', 'btc_usd', 'perpetual', level_size, lookups.okx_trades_lookup),
                "bybit_btcusdt" : tradesflow('bybit', 'btc_usdt', 'perpetual', level_size, lookups.bybit_trades_lookup),
                "bybit_btcusd" : tradesflow('bybit', 'btc_usd', 'perpetual', level_size, lookups.bybit_trades_lookup),
                "bingx_btcusdt" : tradesflow('bingx', 'btc_usdt', 'perpetual', level_size, lookups.bingx_trades_lookup),
                "bitget_btcusdt" : tradesflow('bitget', 'btc_usdt', 'perpetual', level_size, lookups.bitget_trades_lookup),
                "gateio_btcusdt" : tradesflow('gateio', 'btc_usdt', 'perpetual', level_size, lookups.gateio_trades_lookup),
                "htx_btcusdt" : tradesflow('htx', 'btc_usdt', 'perpetual', level_size, lookups.htx_trades_lookup),
                "mexc_btcusdt" : tradesflow('mexc', 'btc_usdt', 'perpetual', level_size, lookups.mexc_trades_lookup),
                "kucoin_btcusdt" : tradesflow('kucoin', 'btc_usdt', 'perpetual', level_size, lookups.kucoin_trades_lookup),
                "deribit_btcusd" : tradesflow('deribit', 'btc_usd', 'perpetual', level_size, lookups.deribit_trades_lookup),
            },
            "oifunding" : {
                "binance_btcusdt" : oiFundingflow('binance', 'btc_usdt', 'perpetual', level_size, lookups.binance_OI_lookup, lookups.binance_funding_lookup),
                "binance_btcusd" : oiFundingflow('binance', 'btc_usd', 'perpetual', level_size, lookups.binance_OI_lookup, lookups.binance_funding_lookup),
                "okx_btcusdt" : oiFundingflow('okx', 'btc_usdt', 'perpetual',level_size, lookups.okx_OI_lookup, lookups.okx_funding_lookup),
                "okx_btcusd" : oiFundingflow('okx', 'btc_usd', 'perpetual',level_size,  lookups.okx_OI_lookup, lookups.okx_funding_lookup),
                "bybit_btcusdt" : oiFundingflow('bybit', 'btc_usdt', 'perpetual', level_size, lookups.bybit_OI_funding_lookup),
                "bybit_btcusd" : oiFundingflow('bybit', 'btc_usd', 'perpetual', level_size, lookups.bybit_OI_funding_lookup),
                "bingx_btcusdt" : oiFundingflow('bingx', 'btc_usdt', 'perpetual', level_size, lookups.bingx_OI_lookup, lookups.bingx_funding_lookup),
                "bitget_btcusdt" : oiFundingflow('bitget', 'btc_usdt', 'perpetual', level_size, lookups.bitget_OI_funding_lookup),
                "gateio_btcusdt" : oiFundingflow('gateio', 'btc_usdt', 'perpetual', level_size, lookups.gateio_OI_lookup, lookups.gateio_funding_lookup),
                "htx_btcusdt" : oiFundingflow('htx', 'btc_usdt', 'perpetual', level_size, lookups.htx_OI_lookup, lookups.htx_funding_lookup),
                "mexc_btcusdt" : oiFundingflow('mexc', 'btc_usdt', 'perpetual', level_size, lookups.mexc_OI_funding_lookup),
                "kucoin_btcusdt" : oiFundingflow('kucoin', 'btc_usdt', 'perpetual', level_size, lookups.kucoin_OI_funding_lookup),
                "deribit_btcusd" : oiFundingflow('deribit', 'btc_usd', 'perpetual', level_size, lookups.deribit_OI_funding_lookup),
            },
            "liquidations" : {
                "binance_btcusdt" : oiFundingflow('binance', 'btc_usdt', 'perpetual', level_size, lookups.binance_liquidations_lookup),
                "binance_btcusd" : oiFundingflow('binance', 'btc_usd', 'perpetual', level_size, lookups.binance_OI_lookup, lookups.binance_funding_lookup),
                "okx_btcusdt" : oiFundingflow('okx', 'btc_usdt', 'perpetual',level_size, lookups.okx_OI_lookup, lookups.okx_funding_lookup),
                "okx_btcusd" : oiFundingflow('okx', 'btc_usd', 'perpetual',level_size,  lookups.okx_OI_lookup, lookups.okx_funding_lookup),
                "bybit_btcusdt" : oiFundingflow('bybit', 'btc_usdt', 'perpetual', level_size, lookups.bybit_OI_funding_lookup),
                "bybit_btcusd" : oiFundingflow('bybit', 'btc_usd', 'perpetual', level_size, lookups.bybit_OI_funding_lookup),
                "gateio_btcusdt" : oiFundingflow('gateio', 'btc_usdt', 'perpetual', level_size, lookups.gateio_OI_lookup, lookups.gateio_funding_lookup),
            },
            "indicators" : {
                "binance_btcusdt" : indicatorflow('binance', 'btc_usdt', 'perpetual', level_size, lookups.binance_GTA_TTA_TTP_lookup),
                "binance_btcusd" : indicatorflow('binance', 'btc_usd', 'perpetual', level_size, lookups.binance_GTA_TTA_TTP_lookup),
                "okx_btc" : indicatorflow('okx', 'btc_usdt', 'perpetual',level_size, lookups.okx_GTA_lookup),
                "bybit_btcusdt" : indicatorflow('bybit', 'btc_usdt', 'perpetual', level_size, lookups.bybit_GTA_lookup),
                "bybit_btcusd" : indicatorflow('bybit', 'btc_usd', 'perpetual', level_size, lookups.bybit_GTA_lookup),
            }
        }
        self.options_axis = {
            "bybit" : oiflowOption('bybit', 'btc', 'opitons', pranges, expiry_windows, lookups.bybit_option_oi_lookup),
            "okx" : oiflowOption('okx', 'btc', 'opitons', pranges, expiry_windows, lookups.okx_option_oi_lookup),
            "deribit" : oiflowOption('deribit', 'btc', 'opitons', pranges, expiry_windows, lookups.deribit_option_oi_lookup),
        }
        ###
        self.data = dict()
        ###

    def merge(self):

        self.spot_books = booksmerger("btc", "perpetual", self.spot_axis["books"])
        self.spot_trades = tradesmerger("btc", "perpetual", self.spot_axis["trades"])
        self.spot_adjustments = booksadjustments("btc", "perpetual", {"books" : self.spot_books, "trades" : self.spot_trades})
        self.perpetual_books = booksmerger("btc", "perpetual", self.perpetual_axis["books"])
        self.perpetual_trades = tradesmerger("btc", "perpetual", self.perpetual_axis["trades"])
        self.perpetual_adjustments = booksadjustments("btc", "perpetual", {"books" : self.perpetual_books, "trades" : self.perpetual_trades})
        self.perpetual_liquidations = lomnifier("btc", "perpetual", self.perpetual_axis["liquidations"])
        self.perpetual_oifunding = oiomnifier("btc", "perpetual", self.perpetual_axis["oifunding"])
        self.perpetual_indicators = indomnifier("btc", "perpetual", "gta", self.perpetual_axis["indicators"])
        self.options = OOImerger("btc", "options", self.expiry_windows, self.pranges, self.options_axis)

        # Books
        self.spot_books.merge_snapshots()
        self.data["spot_books"] = self.spot_books.snapshotO
        self.perpetual_books.merge_snapshots()
        self.data["per_books"] = self.perpetual_books.snapshotO
        # Trades
        self.spot_trades.merge_snapshots()
        self.data["spot_trades"] = self.spot_trades.snapshot
        self.perpetual_trades.merge_snapshots()
        self.data["perp_trades"] = self.perpetual_trades.snapshot
        # Liqs
        self.perpetual_liquidations.merge_snapshots()
        self.data["liqs"] = self.perpetual_liquidations.snapshot
        # Adjustments
        self.spot_adjustments.merge_snapshots()
        self.data["spot_adj"] = self.spot_adjustments.data
        self.perpetual_adjustments.merge_snapshots()
        self.data["perp_adj"] = self.perpetual_adjustments.data
        # OiFunding
        self.perpetual_oifunding.merge_snapshots()
        self.data["oi"] = self.perpetual_oifunding.snapshot
        # Options
        self.options.mergeoi()
        self.data["puts"] = self.options["puts"]
        self.data["calls"] = self.options["calls"]
        # Indicators
        data = self.perpetual_indicators.retrive_data("OIs_per_instrument")
        self.perpetual_indicators.inputOI(data)
        self.perpetual_indicators.inputgta()
        self.data["position"] = self.perpetual_indicators["data"]

    def retrive(self):
        pass

    # Input Depth Spot
    
    def input_binance_spot_btcusdt_depth(self, data):
        self.spot_books["binance_btcusdt"].update_books(data)

    def input_binance_spot_btcfdusd_depth(self, data):
        self.spot_books["binance_btcfdusd"].update_books(data)

    def input_okx_spot_btcusdt_depth(self, data):
        self.spot_books["okx_btcusdt"].update_books(data)

    def input_bybit_spot_btcusdt_depth(self, data):
        self.spot_books["bybit_btcusdt"].update_books(data)

    def input_bybit_spot_btcusdc_depth(self, data):
        self.spot_books["bybit_btcusdc"].update_books(data)

    def input_coinbase_spot_btcusd_depth(self, data):
        self.spot_books["coinbase_btcusd"].update_books(data)

    def input_bingx_spot_btcusdt_depth(self, data):
        self.spot_books["bingx_btcusdt"].update_books(data)

    def input_bitget_spot_btcusdt_depth(self, data):
        self.spot_books["bitget_btcusdt"].update_books(data)

    def input_gateio_spot_btcusdt_depth(self, data):
        self.spot_books["gateio_btcusdt"].update_books(data)

    def input_htx_spot_btcusdt_depth(self, data):
        self.spot_books["htx_btcusdt"].update_books(data)

    def input_mexc_spot_btcusdt_depth(self, data):
        self.spot_books["mexc_btcusdt"].update_books(data)

    def input_kucoin_spot_btcusdt_depth(self, data):
        self.spot_books["kucoin_btcusdt"].update_books(data)

    # Input Trades Spot
    
    def input_binance_spot_btcusdt_trades(self, data):
        self.spot_trades["binance_btcusdt"].input_trades(data)

    def input_binance_spot_btcfdusd_trades(self, data):
        self.spot_bspot_tradesooks["binance_btcfdusd"].input_trades(data)

    def input_okx_spot_btcusdt_trades(self, data):
        self.spot_trades["okx_btcusdt"].input_trades(data)

    def input_bybit_spot_btcusdt_trades(self, data):
        self.spot_trades["bybit_btcusdt"].input_trades(data)

    def input_bybit_spot_btcusdc_trades(self, data):
        self.spot_trades["bybit_btcusdc"].input_trades(data)

    def input_coinbase_spot_btcusd_trades(self, data):
        self.spot_trades["coinbase_btcusd"].input_trades(data)

    def input_bingx_spot_btcusdt_trades(self, data):
        self.spot_trades["bingx_btcusdt"].input_trades(data)

    def input_bitget_spot_btcusdt_trades(self, data):
        self.spot_trades["bitget_btcusdt"].input_trades(data)

    def input_gateio_spot_btcusdt_trades(self, data):
        self.spot_trades["gateio_btcusdt"].input_trades(data)

    def input_htx_spot_btcusdt_trades(self, data):
        self.spot_trades["htx_btcusdt"].input_trades(data)

    def input_mexc_spot_btcusdt_trades(self, data):
        self.spot_trades["mexc_btcusdt"].input_trades(data)

    def input_kucoin_spot_btcusdt_trades(self, data):
        self.spot_trades["kucoin_btcusdt"].input_trades(data)
    
    # Input Depth perp
    
    def input_binance_perp_btcusdt_depth(self, data):
        self.perpetual_axis["books"]["binance_btcusdt"].update_books(data)

    def input_binance_perp_btcusd_depth(self, data):
        self.perpetual_axis["books"]["binance_btcusd"].update_books(data)

    def input_okx_perp_btcusdt_depth(self, data):
        self.perpetual_axis["books"]["okx_btcusdt"].update_books(data)

    def input_okx_perp_btcusd_depth(self, data):
        self.perpetual_axis["books"]["okx_btcusd"].update_books(data)

    def input_bybit_perp_btcusdt_depth(self, data):
        self.perpetual_axis["books"]["bybit_btcusdt"].update_books(data)

    def input_bybit_perp_btcusd_depth(self, data):
        self.perpetual_axis["books"]["bybit_btcusd"].update_books(data)

    def input_bingx_perp_btcusdt_depth(self, data):
        self.perpetual_axis["books"]["bingx_btcusdt"].update_books(data)

    def input_bitget_perp_btcusdt_depth(self, data):
        self.perpetual_axis["books"]["bitget_btcusdt"].update_books(data)

    def input_gateio_perp_btcusdt_depth(self, data):
        self.perpetual_axis["books"]["gateio_btcusdt"].update_books(data)

    def input_htx_perp_btcusdt_depth(self, data):
        self.perpetual_axis["books"]["htx_btcusdt"].update_books(data)

    def input_mexc_perp_btcusdt_depth(self, data):
        self.perpetual_axis["books"]["mexc_btcusdt"].update_books(data)

    def input_kucoin_perp_btcusdt_depth(self, data):
        self.perpetual_axis["books"]["kucoin_btcusdt"].update_books(data)

    def input_deribit_perp_btcusd_depth(self, data):
        self.perpetual_axis["books"]["deribit_btcusd"].update_books(data)

    # Input trades perp
    
    def input_binance_perp_btcusdt_trades(self, data):
        self.perpetual_trades["binance_btcusdt"].input_trades(data)

    def input_binance_perp_btcusd_trades(self, data):
        self.perpetual_trades["binance_btcfdusd"].input_trades(data)

    def input_okx_perp_btcusdt_trades(self, data):
        self.perpetual_trades["okx_btcusdt"].input_trades(data)

    def input_okx_perp_btcusd_trades(self, data):
        self.perpetual_trades["okx_btcusd"].input_trades(data)

    def input_bybit_perp_btcusdt_trades(self, data):
        self.perpetual_trades["bybit_btcusdt"].input_trades(data)

    def input_bybit_perp_btcusd_trades(self, data):
        self.perpetual_trades["bybit_btcusd"].input_trades(data)

    def input_bingx_perp_btcusdt_trades(self, data):
        self.perpetual_trades["bingx_btcusdt"].input_trades(data)

    def input_bitget_perp_btcusdt_trades(self, data):
        self.perpetual_trades["bitget_btcusdt"].input_trades(data)

    def input_gateio_perp_btcusdt_trades(self, data):
        self.perpetual_trades["gateio_btcusdt"].input_trades(data)

    def input_htx_perp_btcusdt_trades(self, data):
        self.perpetual_trades["htx_btcusdt"].input_trades(data)

    def input_mexc_perp_btcusdt_trades(self, data):
        self.perpetual_trades["mexc_btcusdt"].input_trades(data)

    def input_kucoin_perp_btcusdt_trades(self, data):
        self.perpetual_trades["kucoin_btcusdt"].input_trades(data)

    def input_deribit_perp_btcusd_trades(self, data):
        self.perpetual_trades["deribit_btcusd"].input_trades(data)

    # Input oifunding
    
    def input_binance_perp_btcusdt_oi(self, data):
        self.perpetual_oifunding["binance_btcusdt"].input_oi(data)

    def input_binance_perp_btcusdt_fund(self, data):
        self.perpetual_oifunding["binance_btcusdt"].input_funding(data)

    def input_binance_perp_btcusd_oi(self, data):
        self.perpetual_oifunding["binance_btcusd"].input_oi(data)

    def input_binance_perp_btcusd_fund(self, data):
        self.perpetual_oifunding["binance_btcusd"].input_funding(data)

    def input_okx_perp_btcusdt_oi(self, data):
        self.perpetual_oifunding["okx_btcusdt"].input_oi(data)

    def input_okx_perp_btcusdt_fund(self, data):
        self.perpetual_oifunding["okx_btcusdt"].input_funding(data)

    def input_okx_perp_btcusd_oi(self, data):
        self.perpetual_oifunding["okx_btcusd"].input_oi(data)

    def input_okx_perp_btcusd_fund(self, data):
        self.perpetual_oifunding["okx_btcusd"].input_funding(data)

    def input_bybit_perp_btcusdt_oifund(self, data):
        self.perpetual_oifunding["bybit_btcusdt"].input_oi_funding(data)

    def input_bybit_perp_btcusd_oifund(self, data):
        self.perpetual_oifunding["bybit_btcusd"].input_oi_funding(data)

    def input_bingx_perp_btcusdt_oi(self, data):
        self.perpetual_oifunding["bingx_btcusdt"].input_oi(data)

    def input_bingx_perp_btcusdt_fund(self, data):
        self.perpetual_oifunding["bingx_btcusdt"].input_funding(data)

    def input_bitget_perp_btcusdt_oifund(self, data):
        self.perpetual_oifunding["bitget_btcusdt"].input_oi_funding(data)

    def input_gateio_perp_btcusdt_oi(self, data):
        self.perpetual_oifunding["gateio_btcusdt"].input_oi(data)

    def input_gateio_perp_btcusdt_fund(self, data):
        self.perpetual_oifunding["gateio_btcusdt"].input_funding(data)

    def input_htx_perp_btcusdt_fund(self, data):
        self.perpetual_oifunding["htx_btcusdt"].input_funding(data)

    def input_htx_perp_btcusdt_oi(self, data):
        self.perpetual_oifunding["htx_btcusdt"].input_oi(data)

    def input_mexc_perp_btcusdt_oifund(self, data):
        self.perpetual_oifunding["mexc_btcusdt"].input_oi_funding(data)

    def input_kucoin_perp_btcusdt_oifund(self, data):
        self.perpetual_oifunding["kucoin_btcusdt"].input_oi_funding(data)

    def input_deribit_perp_btcusd_oifund(self, data):
        self.perpetual_oifunding["deribit_btcusd"].input_oi_funding(data)
    
    # Input liquidations
    
    def inpu_binance_perp_btcusdt_liq(self, data):
        self.perpetual_liquidations["binance_btcusdt"].input_liquidations(data)

    def inpu_binance_perp_btcusd_liq(self, data):
        self.perpetual_liquidations["binance_btcusd"].input_liquidations(data)

    def inpu_okx_perp_btcusdt_liq(self, data):
        self.perpetual_liquidations["okx_btcusdt"].input_liquidations(data)

    def inpu_okx_perp_btcusd_liq(self, data):
        self.perpetual_liquidations["okx_btcusd"].input_liquidations(data)

    def inpu_bybit_perp_btcusdt_liq(self, data):
        self.perpetual_liquidations["bybit_btcusdt"].input_liquidations(data)

    def inpu_bybit_perp_btcusd_liq(self, data):
        self.perpetual_liquidations["bybit_btcusd"].input_liquidations(data)

    # Input Positions

    def input_binance_btcusdt_gta_tta_ttp(self, data):
        self.perpetual_indicators["binance_btcusdt"].input_binance_gta_tta_ttp(data)

    def input_binance_btcusd_gta_tta_ttp(self, data):
        self.perpetual_indicators["binance_btcusd"].input_binance_gta_tta_ttp(data)
    
    def input_bybit_btcusdt_gta(self, data):
        self.perpetual_indicators["bybit_btcusdt"].input_bybit_gta(data)
    
    def input_bybit_btcusd_gta(self, data):
        self.perpetual_indicators["bybit_btcusd"].input_bybit_gta(data)

    def input_okx_gta(self, data):
        self.perpetual_indicators["okx_btc"].input_okx_gta(data)